In [1]:
from pymongo import MongoClient
import datetime
import re

In [2]:
client = MongoClient()
characters = client.ck2.characters

In [3]:
#wipe the entire collection
characters.delete_many({})

last_char = {}
char = {}
spouse = []

with open("../WorkInProgress2/character.txt", encoding="utf-8") as characterfile:
        for line in characterfile:
            if len(re.findall(r"^\s*([0-9]+)=", line)) > 0:
                #check is b_d is set to prevent inserting first time around. This is because there is no "absolute end" to 
                #a character file, no attribute that is at the end of every character. } is but it appears in a lot of places.
                if("b_d" in char.keys()):
                    try:
                        if len(spouse) > 0:
                            char["spouse"] = spouse                       
                        characters.insert_one(char)
                    except:
                        print("insert failed: " + char)            
                last_char = char
                char = {}
                spouse = []

                #print(re.findall(r"\s*([0-9]+)=", line))
                char["_id"] = int(re.findall(r"\s*([0-9]+)=", line)[0])

            #birth name
            elif len(re.findall(r"\s*bn=\"(.+)\"", line)) > 0:
                char["bn"] = re.findall(r"\s*bn=\"(.+)\"", line)[0]

            #isHistorical
            elif len(re.findall(r"\s*hist=(.+)", line)) > 0:
                char["hist"] = 1

            #isFemale
            elif len(re.findall(r"\s*fem=(.+)", line)) > 0:
                char["fem"] = 1            

            #nickname and date of birth
            elif len(re.findall(r"\s*nick=(\S+)\s+b_d=\"([0-9]+.[0-9]+.[0-9]+)\"", line)) > 0:
                char["nick"] = re.findall(r"\s*nick=(\S+)\s+b_d=\"([0-9]+.[0-9]+.[0-9]+)\"", line)[0][0]
                this_date = re.findall(r"\s*nick=\S+\s+b_d=\"([0-9]+).([0-9]+).([0-9]+)\"", line)
                char["b_d"] = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2])) 
           
            #birth date
            elif len(re.findall(r"\s*.*\s*b_d=\"([0-9]+.[0-9]+.[0-9]+)\"", line)) > 0:
                this_date = re.findall(r"\s*b_d=\"([0-9]+).([0-9]+).([0-9]+)\"", line)
                char["b_d"] = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2]))          

            
            #death date
            elif len(re.findall(r"\s*d_d=\"([0-9]+.[0-9]+.[0-9]+)\"", line)) > 0:
                this_date = re.findall(r"\s*d_d=\"([0-9]+).([0-9]+).([0-9]+)\"", line)                
                char["d_d"] = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2]))
            
            #cause of death
            elif len(re.findall(r"\s*c_d=(.+)", line)) > 0:
                char["c_d"] = re.findall(r"\s*c_d=(.+)", line)[0]

            #killer
            elif len(re.findall(r"\s*killer=([0-9]+)", line)) > 0:
                char["killer"] = int(re.findall(r"\s*killer=([0-9]+)", line)[0])

            #leige
            elif len(re.findall(r"\s*lge=([0-9]+)", line)) > 0:
                char["lge"] = int(re.findall(r"\s*lge=([0-9]+)", line)[0])
                     
            #Real father
            elif len(re.findall(r"\s*rfat=([0-9]+)", line)) > 0:
                char["rfat"] = int(re.findall(r"\s*rfat=([0-9]+)", line)[0])

            #father
            elif len(re.findall(r"\s*fat=([0-9]+)", line)) > 0:
                char["fat"] = int(re.findall(r"\s*fat=([0-9]+)", line)[0])            

            #mother
            elif len(re.findall(r"\s*mot=([0-9]+)", line)) > 0:
                char["mot"] = int(re.findall(r"\s*mot=([0-9]+)", line)[0])           

            #spouse
            elif len(re.findall(r"\s*spouse=([0-9]+)", line)) > 0:
                 spouse.append(int(re.findall(r"\s*spouse=([0-9]+)", line)[0]))

            #religion
            elif len(re.findall(r"\s*rel=\"(.+)\"", line)) > 0:
                char["rel"] = re.findall(r"\s*rel=\"(.+)\"", line)[0]

            #culture
            elif len(re.findall(r"\s*cul=\"(.+)\"", line)) > 0:
                char["cul"] = re.findall(r"\s*cul=\"(.+)\"", line)[0]

            #graphical culture
            elif len(re.findall(r"\s*g_cul=\"(.+)\"", line)) > 0:
                char["g_cul"] = re.findall(r"\s*g_cul=\"(.+)\"", line)[0]

            #dynasty
            elif len(re.findall(r"\s*dnt=([0-9]+)", line)) > 0:
                char["dnt"] = int(re.findall(r"\s*dnt=([0-9]+)", line)[0])

            #isBastard
            elif len(re.findall(r"\s*bstd=(.+)", line)) > 0:
                char["bstd"] = 1

            #employer? (Only set if character is still living)
            elif len(re.findall(r"\s*emp=([0-9]+)", line)) > 0:
                    char["emp"] = int(re.findall(r"\s*emp=([0-9]+)", line)[0])

            #host? (Only set if character is still living)
            elif len(re.findall(r"\s*host=([0-9]+)", line)) > 0:
                    char["host"] = int(re.findall(r"\s*host=([0-9]+)", line)[0])

            #has Bastards
            elif len(re.findall(r"\s*spawned_bastards=(.+)", line)) > 0:
                char["spawned_bastards"] = 1            

#Messy baby! One last insert to get the final character                     
if("b_d" in char.keys()):
    try:
        if len(spouse) > 0:
            char["spouse"] =spouse
        characters.insert_one(char)
    except:
        print("insert failed: " + char)

In [4]:
#add parents and rparents field to use with $graphlookup for full descent. rparents contains rfat if it is set, else fat.
characters.aggregate(
    [
        { "$addFields": { 
            "parents": ["$fat", "$mot"],
            "rparents": [{"$ifNull" : ["$rfat", "$fat"] }, "$mot"] 
        }},
        { "$out": "characters" }
    ]
)

In [5]:
characters.find_one({"_id" : 702356})

{'_id': 702356,
 'b_d': datetime.datetime(1043, 10, 28, 0, 0),
 'bn': 'Cenek',
 'c_d': 'death_trait',
 'd_d': datetime.datetime(1105, 11, 5, 0, 0),
 'dnt': 506,
 'fat': 655400,
 'lge': 730801,
 'mot': 701028,
 'parents': [655400, 701028],
 'rel': 'sunni',
 'rfat': 700388,
 'rparents': [700388, 701028],
 'spouse': [695646, 725517, 749900]}

In [6]:
characters.find_one({"_id" : 641772})

{'_id': 641772,
 'b_d': datetime.datetime(933, 1, 9, 0, 0),
 'bn': 'Böri',
 'c_d': 'death_trait',
 'd_d': datetime.datetime(968, 11, 13, 0, 0),
 'dnt': 10300779,
 'lge': 633087,
 'parents': [None, None],
 'rparents': [None, None]}